In [1]:
import sys
#BASE_DIR = "/home/dzigen/Desktop/ITMO/smiles2024/RAG-project-SMILES-2024-"
BASE_DIR = "/trinity/home/team06/workspace/mikhail_workspace/rag_project"
sys.path.insert(0, BASE_DIR)

In [2]:
from src.Scorer import LLM_SimilarityScorer, SimilarityScorerConfig
from src.Reader import LLM_Model, LLM_Config

/trinity/home/team06/workspace/mikhail_workspace/mikhail_venv/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [3]:
reader_config = LLM_Config()
READER = LLM_Model(reader_config)

Loading checkpoint shards: 100%|██████████| 4/4 [03:28<00:00, 52.16s/it]


In [4]:
scorer_config = SimilarityScorerConfig(
        system_prompt = '''Ты - экспертная система, задача которой - оценить похожесть (смысловую) двух текстов. На вход поступает два текста, твоя задача - вернуть одну из двух меток: "{sim_label}" - тексты содержат одинаковую семантическую информацию (равны по смыслу) и "{dissim_label}" - Тексты содержат разную сементическую информацию (отличаются по смыслу). Если в текстах идёт описание разные объектов/cущностей, то они разные: необходимо поставить метку "{dissim_label}". Не пиши ничего другого, кроме предсказываемой метки релевантности: только "{sim_label}" или "{dissim_label}".

Примеры: 

Текст #1: У меня нет ответа на ваш вопрос.
Текст #2: У меня нет ответа на ваш вопрос.
Метка: {sim_label}

Текст #1: У меня нет ответа на ваш вопрос.
Текст #2: У меня есть ответ на ваш вопрос.
Метка: {dissim_label}

Текст #1: У меня нет ответа на ваш вопрос.
Текст #2: У меня нет соответствующей информации для ответа на ваш вопрос.
Метка: {sim_label}

Текст #1: У меня нет ответа на ваш вопрос.
Текст #2: У меня есть соответствующая информация для ответа на ваш вопрос. 
Метка: {dissim_label}

Текст #1: У меня нет ответа на ваш вопрос.
Текст #2: У меня нет информации, чтобы ответить на ваш вопрос.
Метка: {sim_label}

Текст #1: У меня нет ответа на ваш вопрос.
Текст #2: У меня нет нужной информации для ответа на ваш вопрос.
Метка: {sim_label}

Текст #1: У меня нет ответа на ваш вопрос.
Текст #2: У меня есть нужная информации для ответа на ваш вопрос.
Метка: {dissim_label}

Текст #1: У меня нет ответа на ваш вопрос.
Текст #2: У меня нет необходимой информации для ответа на ваш вопрос.
Метка: {sim_label}

Текст #1: У меня нет ответа на ваш вопрос.
Текст #2: У меня есть информации для ответа на ваш вопрос.
Метка: {dissim_label}

Текст #1: У меня нет ответа на ваш вопрос.
Текст #2: Чай будет вкуснее, если его нагреть и добавить сахар.
Метка: {dissim_label}

Текст #1: У меня нет ответа на ваш вопрос.
Текст #2: Дельфины живут в больших водоёмах.
Метка: {dissim_label}''',
    user_prompt_template = "Текст #1: {txt1}\nТекст #2: {txt2}\nМетка: "
)

In [5]:
SCORER = LLM_SimilarityScorer(scorer_config, READER)

In [6]:
examples = [
    ['Я могу ответить на твой вопрос', 'У меня нет необходимой информации для ответа на твой вопрос'],
    ['У меня нет ответа на твой вопрос', 'У меня есть ответ на вопрос'],
    ['Дельфины живут в воду', "Чай вкуснее, если он горячий"],
    ["Когда родился Александр Сергеевич Пушкин?", "Ф. М. Достоевский родился 30 октября (11 ноября) 1821 г. в семье врача московской Мариинской больницы для бедных Михаила Андреевича Достоевского."],
    ["Кто написал книгу 'Атлант расправил плечи'?", "«Атла́нт расправил плечи» (англ. Atlas Shrugged, дословно — «Атлант пожал плечами») — роман-антиутопия американской писательницы Айн Рэнд, впервые опубликованный в 1957 году. Является четвёртым и последним романом Рэнд, а также самым длинным."]
]

In [7]:
for i, example_pair in enumerate(examples):
    print("\nEXAMPLE #1")
    print(f"TEXT 1:\n{example_pair[0]}")
    print(f"TEXT 2:\n{example_pair[1]}")
    
    predicted_score = SCORER.predict(example_pair[0], [example_pair[1]])
    print(f"SCORE:\n{predicted_score[0]}")


EXAMPLE #1
TEXT 1:
Я могу ответить на твой вопрос
TEXT 2:
У меня нет необходимой информации для ответа на твой вопрос
SCORE:
разные

EXAMPLE #1
TEXT 1:
У меня нет ответа на твой вопрос
TEXT 2:
У меня есть ответ на вопрос
SCORE:
разные

EXAMPLE #1
TEXT 1:
Дельфины живут в воду
TEXT 2:
Чай вкуснее, если он горячий
SCORE:
разные

EXAMPLE #1
TEXT 1:
Когда родился Александр Сергеевич Пушкин?
TEXT 2:
Ф. М. Достоевский родился 30 октября (11 ноября) 1821 г. в семье врача московской Мариинской больницы для бедных Михаила Андреевича Достоевского.
SCORE:
похожие

EXAMPLE #1
TEXT 1:
Кто написал книгу 'Атлант расправил плечи'?
TEXT 2:
«Атла́нт расправил плечи» (англ. Atlas Shrugged, дословно — «Атлант пожал плечами») — роман-антиутопия американской писательницы Айн Рэнд, впервые опубликованный в 1957 году. Является четвёртым и последним романом Рэнд, а также самым длинным.
SCORE:
похожие
